In [ ]:
from google.colab import drive    #montei o drive
drive.mount('/content/drive', force_remount=True) #tinha bugado do jeito normal por isso montei assim

Mounted at /content/drive


In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login()

In [ ]:
wandb.init(
  project="UNET",
)

In [ ]:
#importando as bibliotecas
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF
import os
from PIL import Image
import glob
from torch.utils.data import Dataset
import numpy as np
from albumentations.pytorch import ToTensorV2
import albumentations as A
import torch.optim as optim
#from tqdm import tqdm
from tqdm.notebook import tqdm_notebook as tqdm
from torch.utils.data.dataloader import DataLoader
import torchvision
import random
import shutil
from sklearn.metrics import jaccard_score
import torch.nn.functional as F
import matplotlib.pyplot as plt

In [ ]:
#aqui é estrutura padrão da UNET
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False), #bias = falso porque o batchnorm vai cancelar ele
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )
    def forward(self, x):
        return self.conv(x)

#arquiterura
class UNET(nn.Module):
    def __init__(
            self, in_channels=3, out_channels=1, features=[64, 128, 256, 512],
    ):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        #fazendo a parte de baixo da arquitetura aqui
        for feature in features:  #para cada valor feature na lista uma camada é adicionada a lista
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        #parte de cima da arquitetura
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1]

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]
            #se não der pra concatenar devido ao tamanho da entrada

            if x.shape != skip_connection.shape:
                 x = TF.resize(x, size=skip_connection.shape[2:], antialias=None)   #Antialias = None porque vai atualizar e mudar o modo padrão para True então manter em None

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)


        return self.final_conv(x)



def test():
    x = torch.rand((3, 1, 161, 161))
    model = UNET(in_channels=1, out_channels=1)
    preds = model(x)
    print(preds.shape)
    print(x.shape)
    assert preds.shape == x.shape

if __name__ == "__main__":
    test()

In [ ]:
class StrawberryDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
      return len(self.images)

    def __getitem__(self, index):
        img_path = os.path.join(self.image_dir, self.images[index].replace(".jpg", ".png")) #parte do replace é teste
        mask_path = os.path.join(self.mask_dir, self.images[index].replace(".jpg", ".png"))
        image = np.array(Image.open(img_path).convert("RGB"))
        mask = np.array(Image.open(mask_path).convert("L"), dtype=np.float32)
        mask[mask ==255.0] = 1.0

        if self.transform is not None:
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]

        return image, mask

In [ ]:
#salvando
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)

def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])


def get_loaders(
    train_dir,
    train_maskdir,
    val_dir,
    val_maskdir,
    batch_size,
    train_transform,
    val_transform,
    num_workers=4,
    pin_memory=True,
):
    train_ds  = StrawberryDataset(
        image_dir=train_dir,
        mask_dir=train_maskdir,
        transform=train_transform,
    )

    train_loader = DataLoader(
        train_ds,
        batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True,
    )

    val_ds = StrawberryDataset(
        image_dir=val_dir,
        mask_dir=val_maskdir,
        transform=val_transform,
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False,
    )

    return train_loader, val_loader


#analise dos resultados
def iou_score(preds, targets, smooth=1e-6):
    assert preds.shape == targets.shape, "As dimensões de preds e targets devem ser iguais."

    preds = preds > 0.5
    targets = targets > 0.5

    intersection = (preds * targets).sum()
    union = preds.sum() + targets.sum() - intersection
    iou = (intersection) / (union + smooth)

    return iou


def dice_score(preds, targets, smooth=1e-6):
    intersection = torch.logical_and(preds, targets).sum()
    union = preds.sum() + targets.sum()
    dice = (2 * intersection + smooth) / (union + smooth)
    return dice

def recall(preds, targets, smooth=1e-6):
   #binariza as previsões e os alvos
    preds = preds > 0.5
    targets = targets > 0.5

    # calcula o número de verdadeiros positivos e falsos negativos
    true_positives = torch.logical_and(preds, targets).sum()
    false_negatives = targets.sum() - true_positives

     # calcula o recall com suavização para evitar divisão por zero
    recall_value = true_positives / (true_positives + false_negatives + smooth)
    return recall_value

def falso_positivo(preds, targets, smooth=1e-6):
    #binarizo as previsões e alvos
    preds = preds > 0.5 #qualquer valor acima de 0.5 é positivo e abaixo de 0.5 é negativo
    targets = targets > 0.5

    falsos_positivos = (preds > targets).sum()  #calcula o número de falsos positivos

    verdadeiros_negativos = (~preds & ~targets).sum()  #uso ~como negativo, ou seja quando ambos são 0, tanto a máscara como a predição.

    fpr = falsos_positivos.sum() / (falsos_positivos.sum() + verdadeiros_negativos.sum() + smooth)    #fórmula + smooth para evitar divisão por 0
    return fpr

def falso_negativo(preds, targets, smooth=1e-6):
    #binarizo as previsões e alvos
    preds = preds > 0.5 #qualquer valor acima de 0.5 é positivo e abaixo de 0.5 é negativo
    targets = targets > 0.5

    verdadeiros_positivos = torch.logical_and(targets, preds).sum() #usa essa função lógica do pytorch para calcular os acertos

    falsos_negativos = targets.sum() - verdadeiros_positivos # aqui faço:  o total de positivos certo - o total de positivos acertados = o número de positivos que foi predito como negativos

    fnr = falsos_negativos.sum() / (falsos_negativos.sum() + verdadeiros_positivos.sum() + smooth) #fórmula + smooth para evitar divisão por 0

    return fnr

#função que calcula as métricas todas
def check_accuracy(loader, model, device="cuda"):
    #incializo todas métricas em 0
    num_correct = 0
    num_pixels = 0
    dice_score_total = 0
    iou = 0.0
    recall_total = 0.0
    fpr_total = 0.0
    fnr_total = 0.0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device).unsqueeze(1)
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()

             # contagem de pixels corretos
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)

            # cálculo das métricas
            dice_score_total += dice_score(preds, y)
            iou += iou_score(preds, y)
            recall_total += recall(preds, y)
            fpr_total += falso_positivo(preds, y)
            fnr_total += falso_negativo(preds, y)

    """Métricas no wandb"""
    wandb.log({"acc": num_correct / num_pixels, "dice_score": dice_score_total / len(loader), "iou_score": iou / len(loader), "recall": recall_total / len(loader)})
    wandb.log({"Falso positivo": fpr_total / len(loader), "Falso Negativo": fnr_total / len(loader)})

    print(f"Na Validação \n Obteve: {num_correct}/{num_pixels} with acc {num_correct / num_pixels * 100:.2f}")
    print(f"Dice Score: {dice_score_total / len(loader)}")
    print(f"IoU Score: {iou / len(loader)}")
    print(f"Recall: {recall_total / len(loader)}")
    print(f"Taxa de falsos positivos: {fpr_total/ len(loader)}")
    print(f"Taxa de falsos Negativos: {fnr_total/ len(loader)}")

    model.train()

def save_predictions_as_imgs(
    loader, model, folder="/content/drive/MyDrive/saved_images/", device="cuda"
):
    model.eval()
    for idx, (x, y) in enumerate (loader):
        x = x.to(device=device)
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()


        torchvision.utils.save_image(
            preds, f"{folder}/pred_{idx}.png"
        )
        torchvision.utils.save_image(y.unsqueeze(1), f"{folder}/true_{idx}.png")
    model.train()

In [ ]:
#parâmetros padrã0
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 8
NUM_EPOCHS = 60
NUM_WORKERS = 2
IMAGE_HEIGHT = 160
IMAGE_WIDTH = 240
PIN_MEMORY = True
LOAD_MODEL = False    #carregar modelo treinado
#diretório das imagens de treino
TRAIN_IMG_DIR = "/content/drive/MyDrive/fulldividido/treino /images/"
 #diretório das máscaras de treino
TRAIN_MASK_DIR = "/content/drive/MyDrive/fulldividido/treino /masks/"
#diretório das imagens de teste
VAL_IMG_DIR = "/content/drive/MyDrive/fulldividido/teste/images/"
#diretório das máscaras de teste
VAL_MASK_DIR = "/content/drive/MyDrive/fulldividido/teste/masks/"

#padrão pytorch de treino
def train_fn(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader)

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)

        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)
            wandb.log({"loss": loss})     #carrega loss no wandb

        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        #atualizando a loss no loop do tqdm
        loop.set_postfix(loss=loss.item())




def main():
    train_transform = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Rotate(limit=35, p=0.3),                        #Rotaciona em até 35 graus
            A.HorizontalFlip(p=1.0),                          #Espelha horizontalmente com probabilidade de 0,5
            A.CLAHE(p=0.2),                                   #ajuda a melhorar a visibilidade de detalhes em regiões escuras ou claras da imagem.
            A.Normalize(                   #normaliza os valores de pixel da imagem para ter média 0 e desvio padrão 1
                mean=[0.5, 0.5, 0.5],
                std=[0.5, 0.5, 0.5],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),     #converte a imagem para o formato tensor
        ],
    )

    val_transforms = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),   #redimensiona a imagem
            A.Normalize(
                mean=[0.5, 0.5, 0.5],
                std=[0.5, 0.5, 0.5],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )



    model = UNET(in_channels=3, out_channels=1).to(DEVICE)
    loss_fn = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


    train_loader, val_loader = get_loaders(
        TRAIN_IMG_DIR,
        TRAIN_MASK_DIR,
        VAL_IMG_DIR,
        VAL_MASK_DIR,
        BATCH_SIZE,
        train_transform,
        val_transforms,
        NUM_WORKERS,
        PIN_MEMORY

    )



    scaler = torch.cuda.amp.GradScaler()

    for epoch in range(NUM_EPOCHS):
        train_fn(train_loader, model, optimizer, loss_fn, scaler)

        #salvar modelo
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer":optimizer.state_dict(),
        }
        save_checkpoint(checkpoint)

        #checar a acurácia da validação
        check_accuracy(val_loader, model, device=DEVICE)


        #printa uns exemplos gerados
        save_predictions_as_imgs(
            val_loader, model, folder="/content/drive/MyDrive/saved_images/", device=DEVICE  #saved_images/
        )


if __name__ =="__main__":
    main()




  0%|          | 0/54 [00:00<?, ?it/s]

=> Saving checkpoint
Na Validação 
 Obteve: 5512591/7104000 with acc 77.60
Dice Score: 0.6085001230239868
IoU Score: 0.6776763200759888
Recall: 0.8321400880813599
Taxa de falsos positivos: 0.1793353259563446
Taxa de falsos Negativos: 0.1678600162267685


  0%|          | 0/54 [00:00<?, ?it/s]

=> Saving checkpoint
Na Validação 
 Obteve: 5830875/7104000 with acc 82.08
Dice Score: 0.6494271159172058
IoU Score: 0.744285523891449
Recall: 0.9035264849662781
Taxa de falsos positivos: 0.1840483844280243
Taxa de falsos Negativos: 0.09647362679243088


  0%|          | 0/54 [00:00<?, ?it/s]

...

=> Saving checkpoint
Na Validação 
 Obteve: 6181925/7104000 with acc 87.02
Dice Score: 0.6772708296775818
IoU Score: 0.808682382106781
Recall: 0.9195126891136169
Taxa de falsos positivos: 0.11262088268995285
Taxa de falsos Negativos: 0.08048743009567261


  0%|          | 0/54 [00:00<?, ?it/s]

=> Saving checkpoint
Na Validação 
 Obteve: 6240168/7104000 with acc 87.84
Dice Score: 0.6773130893707275
IoU Score: 0.8164575099945068
Recall: 0.9136931300163269
Taxa de falsos positivos: 0.0968678742647171
Taxa de falsos Negativos: 0.08630697429180145


  0%|          | 0/54 [00:00<?, ?it/s]

=> Saving checkpoint
Na Validação 
 Obteve: 6294408/7104000 with acc 88.60
Dice Score: 0.6797139644622803
IoU Score: 0.8256763815879822
Recall: 0.9188864231109619
Taxa de falsos positivos: 0.09487804025411606
Taxa de falsos Negativos: 0.08111347258090973
